# SimSwap
Reference: https://github.com/neuralchen/SimSwap

## Prepare code

### SimSwap

In [ ]:
%cd /content
!git clone https://github.com/woctezuma/SimSwap.git
%cd /content/SimSwap/
!git checkout upgrade-insightface

In [ ]:
!pip install insightface==0.7.3 onnxruntime moviepy > /dev/null
!pip install imageio==2.34.0 > /dev/null

### (optional) GPEN

In [ ]:
%cd /content
!git clone https://github.com/yangxy/GPEN.git

# Remove the example input
%rm -rf /content/GPEN/examples
%mkdir -p /content/GPEN/examples/imgs

In [ ]:
%pip install ninja

## Prepare models

### SimSwap

In [ ]:
%cd /content/SimSwap

In [ ]:
!wget https://github.com/woctezuma/SimSwap-colab/releases/download/antelope/antelope.zip
!wget -P ./arcface_model https://github.com/woctezuma/SimSwap-colab/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth
!wget https://github.com/neuralchen/SimSwap/releases/download/512_beta/512.zip

In [ ]:
!unzip ./checkpoints.zip  -d ./checkpoints

!unzip 512.zip -d ./checkpoints

!unzip antelope.zip -d ./insightface_func/models/

### (optional) GPEN

In [ ]:
%cd /content/GPEN/weights

url = "https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models"
OSSAccessKeyId = "LTAI4G6bfnyW4TA4wFUXTYBe"

model_name = "RetinaFace-R50.pth"
Expires = "1961116085"
Signature = "GlUNW6%2B8FxvxWmE9jKIZYOOciKQ%3D"
!wget "{url}/{model_name}?OSSAccessKeyId={OSSAccessKeyId}&Expires={Expires}&Signature={Signature}" -O {model_name}

model_name = "GPEN-BFR-512.pth"
Expires = "1961116208"
Signature = "hBgvVvKVSNGeXqT8glG%2Bd2t2OKc%3D"
!wget "{url}/{model_name}?OSSAccessKeyId={OSSAccessKeyId}&Expires={Expires}&Signature={Signature}" -O {model_name}

model_name = "realesrnet_x1.pth"
Expires = "1968049923"
Signature = "omV%2Fb8Jibkgl1FggsR%2B821jQvOI%3D"
!wget "{url}/{model_name}?OSSAccessKeyId={OSSAccessKeyId}&Expires={Expires}&Signature={Signature}" -O {model_name}

model_name = "ParseNet-latest.pth"
Expires = "1961116134"
Signature = "bnMwU1JogmNbARto6G%2B7iaJQCHs%3D"
!wget "{url}/{model_name}?OSSAccessKeyId={OSSAccessKeyId}&Expires={Expires}&Signature={Signature}" -O {model_name}

## Prepare data

### Download

In [ ]:
%cd /content

!curl https://i.imgur.com/iQtmj1N.png -o timsweeney.png
!curl https://i.imgur.com/ujuUXmg.png -o alanwake.png

In [ ]:
input_fname = '/content/timsweeney.png'
output_fname = '/content/alanwake.png'

### Convert to JPG

Images should not be too large, hence the (arbitrary) limitation of 1024 length.

In [ ]:
def get_new_size(img_size,
                 max_allowed_length = 1024):

  if any(max_allowed_length < sz for sz in img_size):
    long_length = max(img_size)
    ratio = max_allowed_length / long_length
  else:
    ratio = 1.0

  new_img_size = [
                  int(ratio*sz)
                  for sz in img_size
                  ]

  return tuple(new_img_size)

In [ ]:
from PIL import Image

allow_resize = False

for fname in [input_fname, output_fname]:
  jpg_fname = fname.replace('.png', '.jpg')

  try:
    img = Image.open(fname)
  except FileNotFoundError:
    continue

  new_size = get_new_size(img.size, max_allowed_length = 1024)
  if allow_resize:
    print(f'Resizing from {img.size} to {new_size}')
    img.resize(new_size)

  print(f'Saving to {jpg_fname}')
  img.convert('RGB').save(jpg_fname)

jpg_input = input_fname.replace('.png', '.jpg')
jpg_output = output_fname.replace('.png', '.jpg')

## Run SimSwap

### Single

In [ ]:
%cd /content/SimSwap
%mkdir -p /content/output/single/

!python test_wholeimage_swapsingle.py \
 --no_simswaplogo \
 --use_mask \
 --crop_size 512 \
 --isTrain false  --name people \
 --Arc_path arcface_model/arcface_checkpoint.tar \
 --pic_a_path {jpg_input} \
 --pic_b_path {jpg_output} \
 --output_path /content/output/single/ > /dev/null


### Multi

In [ ]:
%cd /content/SimSwap
%mkdir -p /content/output/multi/

!python test_wholeimage_swapmulti.py \
 --no_simswaplogo \
 --use_mask \
 --crop_size 512 \
 --isTrain false  --name people \
 --Arc_path arcface_model/arcface_checkpoint.tar \
 --pic_a_path {jpg_input} \
 --pic_b_path {jpg_output} \
 --output_path /content/output/multi/ > /dev/null


## (optional) Post-process with GPEN

In [ ]:
%cp /content/output/single/result_whole_swapsingle.jpg /content/GPEN/examples/imgs/
%cp /content/output/multi/result_whole_swapmulti.jpg /content/GPEN/examples/imgs/

In [ ]:
%cd /content/GPEN
!python demo.py --sr_scale 1